## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [4]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_counts)

# Replace in DataFrame
replace_application = application_counts[application_counts < 10].index
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(replace_application, 'Other')

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 10
application_types_to_replace = application_counts[application_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure replacement was successful
print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: count, dtype: int64


In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

# Replace in DataFrame
replace_classification = classification_counts[classification_counts < 1000].index
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(replace_classification, 'Other')

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_greater_than_one = classification_counts[classification_counts > 1]
print(classification_counts_greater_than_one)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure replacement was successful
print(application_df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [11]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Third hidden layer (new)
nn.add(tf.keras.layers.Dense(units=25, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4800      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 25)                1275      
                                                                 
 dense_3 (Dense)             (None, 1)                 26        
                                                                 
Total params: 11151 (43.56 KB)
Trainable params: 11151 (43.56 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
# Train the model with more epochs
history = nn.fit(X_train_scaled, y_train, epochs=200, batch_size=32, validation_split=0.2)

Epoch 1/200
686/686 [==============================] - 3s 3ms/step - loss: 0.5330 - accuracy: 0.7391 - val_loss: 0.5629 - val_accuracy: 0.7385
Epoch 2/200
686/686 [==============================] - 2s 3ms/step - loss: 0.5323 - accuracy: 0.7401 - val_loss: 0.5648 - val_accuracy: 0.7394
Epoch 3/200
686/686 [==============================] - 3s 5ms/step - loss: 0.5322 - accuracy: 0.7397 - val_loss: 0.5678 - val_accuracy: 0.7374
Epoch 4/200
686/686 [==============================] - 2s 3ms/step - loss: 0.5318 - accuracy: 0.7389 - val_loss: 0.5718 - val_accuracy: 0.7365
Epoch 5/200
686/686 [==============================] - 2s 3ms/step - loss: 0.5331 - accuracy: 0.7392 - val_loss: 0.5711 - val_accuracy: 0.7372
Epoch 6/200
686/686 [==============================] - 2s 3ms/step - loss: 0.5319 - accuracy: 0.7400 - val_loss: 0.5703 - val_accuracy: 0.7387
Epoch 7/200
686/686 [==============================] - 2s 3ms/step - loss: 0.5321 - accuracy: 0.7402 - val_loss: 0.5723 - val_accuracy: 0.7372

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.6038 - accuracy: 0.7257 - 490ms/epoch - 2ms/step
Loss: 0.6038388013839722, Accuracy: 0.7256559729576111


In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=256, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               12288     
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 53505 (209.00 KB)
Trainable params: 53505 (209.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Compile the model with a lower learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [28]:
# Train the model with more epochs
history = nn.fit(X_train_scaled, y_train, epochs=300, batch_size=32, validation_split=0.2)

Epoch 1/300
686/686 [==============================] - 5s 5ms/step - loss: 0.5705 - accuracy: 0.7195 - val_loss: 0.5521 - val_accuracy: 0.7385
Epoch 2/300
686/686 [==============================] - 3s 4ms/step - loss: 0.5564 - accuracy: 0.7294 - val_loss: 0.5514 - val_accuracy: 0.7382
Epoch 3/300
686/686 [==============================] - 2s 3ms/step - loss: 0.5535 - accuracy: 0.7296 - val_loss: 0.5497 - val_accuracy: 0.7407
Epoch 4/300
686/686 [==============================] - 2s 3ms/step - loss: 0.5513 - accuracy: 0.7297 - val_loss: 0.5456 - val_accuracy: 0.7389
Epoch 5/300
686/686 [==============================] - 2s 3ms/step - loss: 0.5500 - accuracy: 0.7305 - val_loss: 0.5472 - val_accuracy: 0.7385
Epoch 6/300
686/686 [==============================] - 3s 4ms/step - loss: 0.5484 - accuracy: 0.7308 - val_loss: 0.5494 - val_accuracy: 0.7371
Epoch 7/300
686/686 [==============================] - 3s 4ms/step - loss: 0.5480 - accuracy: 0.7336 - val_loss: 0.5461 - val_accuracy: 0.7387

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.7387 - accuracy: 0.7258 - 455ms/epoch - 2ms/step
Loss: 0.7387076616287231, Accuracy: 0.7258017659187317


In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=256, activation='selu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='selu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='selu'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation='selu'))

# Output Layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 256)               12288     
                                                                 
 dense_14 (Dense)            (None, 128)               32896     
                                                                 
 dense_15 (Dense)            (None, 64)                8256      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 1)                 33        
                                                                 
Total params: 55553 (217.00 KB)
Trainable params: 55553 (217.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
# Compile the model with a lower learning rate
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)
nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [54]:
# Train the model with more epochs
history = nn.fit(X_train_scaled, y_train, epochs=500, batch_size=64, validation_split=0.2)

Epoch 1/500
343/343 [==============================] - 2s 6ms/step - loss: 0.5426 - accuracy: 0.7350 - val_loss: 0.5477 - val_accuracy: 0.7305
Epoch 2/500
343/343 [==============================] - 2s 6ms/step - loss: 0.5424 - accuracy: 0.7342 - val_loss: 0.5462 - val_accuracy: 0.7376
Epoch 3/500
343/343 [==============================] - 3s 8ms/step - loss: 0.5421 - accuracy: 0.7346 - val_loss: 0.5463 - val_accuracy: 0.7385
Epoch 4/500
343/343 [==============================] - 2s 7ms/step - loss: 0.5417 - accuracy: 0.7352 - val_loss: 0.5462 - val_accuracy: 0.7362
Epoch 5/500
343/343 [==============================] - 2s 5ms/step - loss: 0.5415 - accuracy: 0.7351 - val_loss: 0.5452 - val_accuracy: 0.7382
Epoch 6/500
343/343 [==============================] - 2s 5ms/step - loss: 0.5411 - accuracy: 0.7360 - val_loss: 0.5480 - val_accuracy: 0.7380
Epoch 7/500
343/343 [==============================] - 2s 6ms/step - loss: 0.5407 - accuracy: 0.7364 - val_loss: 0.5443 - val_accuracy: 0.7392

In [55]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 1.4683 - accuracy: 0.7265 - 384ms/epoch - 2ms/step
Loss: 1.4683467149734497, Accuracy: 0.7265306115150452


In [56]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization_1.h5')